# Pytorch tutorial
*Adapted from [Deep Learning with PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html "Pytorch tutorial")*

Pytorch is:  
* A deep learning framework (mainly geared towards research)  
* A GPU-powered numpy   

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## Tensors

Differently from Numpy ndarrays, Tensors can be used on a GPU

In [ ]:
x = torch.ones(2,4, dtype=torch.float)
print(x)

In [ ]:
x = torch.rand(2,4)
print(x)

In [ ]:
x = torch.tensor([[1.2, 4],[3, 5.6]])
print(x)

Tensors can be easily manipulated using usual operation

In [ ]:
print(x.size())

y = torch.rand(2,2)
print(x + y)
# alternatively
z =  torch.empty(2,2)
torch.add(x,y, out=z)
print(z)

In [ ]:
print(z.size())

# to reshape a multi-dimensional array
z = z.view(4)     # the size -1 can be used, the dimension is inferred from the others
print(z.size())

From Numpy to Pytorch variables share the memory location, hence we can easily **modify** them, as long as they are on CPU

In [ ]:
a = torch.ones(3,2)
print(a)

# from Pytorch to Numpy
b = a.numpy()
print(b)
a.add_(3)
print(a)
print(b)

In [ ]:
# from Numpy to Pytorch
c = torch.from_numpy(b)
print(c)

## Automatic differentiation

The ``autograd`` package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

In [ ]:
x = torch.ones(2,4, dtype=torch.float)
print(x)
print(x.requires_grad)

In [ ]:
x.requires_grad_(True)
print(x)
print(x.requires_grad)

In [ ]:
y = x + 7
print(y)

``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor``.
Other important methods:  
* ``.backward()``: gradients are computed automatically
* ``.detach()``: future computations are not tracked

We can also prevent tracking history wrapping the code in a block in ``with torch.no_grad():``

## Neural networks

We make use of the ``torch.nn`` package. An ``nn.Module`` contains layers, and a method ``forward(input)`` that returns the ``output``

### Checklist for neural networks
1. Define the neural network that has some learnable parameters (or
  weights)
2. Iterate over a dataset of inputs, for each input:
  1. Process input through the network
  2. Compute the loss (how far is the output from being correct)
  3. Propagate gradients back into the network’s parameters
  4. Update the weights of the network, using some stochastic gradient descent algorithm

#### 1. Example of network definition and initialization

In [ ]:
class Net_MNIST(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1):
        super(Net_MNIST, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.output_dim = output_dim
        
        self.fc1 = nn.Linear(self.input_dim, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
    
# create a Net object, i.e. our neural network
input_dim = 28**2   # assuming as input an 28x28 image
hidden_dim = 30
batch_size = 1
output_dim = 10

net = Net_MNIST(input_dim=input_dim, hidden_dim=hidden_dim, batch_size=batch_size, output_dim=output_dim)
print(net)

For each mini-batch of data:

#### A. Processing input
The network is called as a function, using the input batch data to get the output batch

In [ ]:
inp = torch.randn(batch_size, 28, 28)
out = net(inp.view(-1,input_dim))

print(out)

#### B. Computing loss
There are a number of pre-defined loss functions, but it is possible to define our own.
It is important that, given the prediction $y_{pred} \in \mathbb{R}^d$ and the ground truth $y_{true} \in \mathbb{R}^d$, the loss $\mathcal{L}$ is defined so that

$$ \mathcal{L} : \mathbb{R}^d \times \mathbb{R}^d \to \mathbb{R}$$

In [ ]:
target = torch.randn(10)         # a dummy target, for example
target = target.view(1, -1)      # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

#### C. Backpropagation
The computation of the gradient is done automatically using the ``.backward()`` method

#### D. Update the weights
We need to choose an optimization algorithm, different options are available as well

In [ ]:
# create the optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in the training loop:
optimizer.zero_grad()             # zero the gradient buffers
out = net(inp.view(-1,input_dim))
loss = criterion(out, target)
loss.backward()
optimizer.step()                  # does the update